<h1>Primo Progetto<h1>

In [1]:
# Importo le librerie richieste
import pandas as pd
from serpapi import GoogleScholarSearch
import json

In [2]:
GoogleScholarSearch.SERP_API_KEY = "07e028babd5fee852e28e67303bacde47f6d4179bafb94b53f313f89b17409e8"

#### 1 | Utilizzo della libreria Python SerpAPI per scaricare, per ciascuno dei 7 autori elencati nel file nodes.csv

In [3]:
# Importo il file nodes.csv
df = pd.read_csv('data/nodes.csv')
display(df)

,name,affiliations
0,David La Barbera,Università Degli Studi Di Udine
1,Michael Soprano,Postdoctoral Research Fellow at the University...
2,Kevin Roitero,University of Udine
3,Stefano Mizzaro,Full professor of Computer Science and Informa...
4,Damiano Spina,"School of Computing Technologies, RMIT University"
5,Gianluca Demartini,Associate Professor at the University of Queen...
6,Eddy Maddalena,"Università degli Studi di Udine, Italy"


In [4]:

updated_df = pd.DataFrame(columns=['author_id', 'cited_by', 'interests'])

# Itero su ogni autore del dataframe
for index, row in df.iterrows():
    author_name = row['name']
    author_affliation = row['affiliations']
        
    params = {
        "engine" : "google_scholar_profiles",
        "hl": "en",
        "mauthors":f'{author_name},{author_affliation}'
    }
    search = GoogleScholarSearch(params)
    results = search.get_dict()['profiles']
    
    for profile in results:
        author_id = profile['author_id']
        cited_by = profile['cited_by']
        
        interests = []
        
        for interest in profile.get('interests', ):
            title = interest.get('title', '')
            interests.append(title)
    
    interests_str = ', '.join(interests)
    
    # Aggiungo una nuova riga al dataframe updated_df
    new_row = {'author_id': author_id, 'cited_by': cited_by, 'interests': interests_str}
    updated_df = pd.concat([updated_df, pd.DataFrame([new_row])], ignore_index=True)

new_df = pd.merge(df, updated_df, left_index=True, right_index=True)
display(new_df)



,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,139,Computer Science
1,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,210,"Computer Science, Crowdsourcing"
2,Kevin Roitero,University of Udine,1xd52jMAAAAJ,690,"Artificial Intelligence, Crowdsourcing and Hum..."
3,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4739,"Information retrieval, mobile systems, scholar..."
4,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2483,"Information Retrieval, Text Analytics, Data Sc..."
5,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,5348,"Information Retrieval, Semantic Web, Human Com..."
6,Eddy Maddalena,"Università degli Studi di Udine, Italy",egHDpFkAAAAJ,553,"Information Retrieval, Crowdsourcing, Data Sci..."


In [5]:
# Sovrascrivo il file nodes.csv con le nuove informazioni trovate
new_df.to_csv('data/nodes.csv', index=False)

#### 2a

In [6]:
# Carica il file CSV aggiornato con gli ID degli autori
df = pd.read_csv('data/nodes.csv')

# Crea un DataFrame vuoto per i coautori
coauthors_df = pd.DataFrame(columns=['name', 'affiliations', 'author_id', 'cited_by', 'interests'])

# Itera su ogni autore
for index, row in df.iterrows():
    author_id = row['author_id']
    
    # Recupera il profilo dell'autore utilizzando l'ID
    author_params = {
        "engine": "google_scholar_author",
        "author_id": author_id,
        "hl": "en"
    }
    author_search = GoogleScholarSearch(author_params)
    author_profile = author_search.get_dict()

    # Elenca i coautori
    coauthors = author_profile.get('co_authors', [])

    # Per ogni coautore, esegui una ricerca separata
    for coauthor in coauthors:
        coauthor_name = coauthor['name']

        # Ricerca il profilo del coautore
        coauthor_params = {
            "engine": "google_scholar_profiles",
            "hl": "en",
            "mauthors": coauthor_name
        }
        coauthor_search = GoogleScholarSearch(coauthor_params)
        coauthor_results = coauthor_search.get_dict()['profiles'][0]  # Assumendo il primo profilo come corretto

        # Crea una riga con le informazioni del coautore
        new_row = pd.DataFrame([{
            'name': coauthor_results['name'],
            'affiliations': coauthor_results.get('affiliations', ''),
            'author_id': coauthor_results.get('author_id', ''),
            'cited_by': coauthor_results.get('cited_by', ''),
            'interests': ', '.join([interest['title'] for interest in coauthor_results.get('interests', [])])
        }])

        # Aggiunge la nuova riga al DataFrame
        coauthors_df = pd.concat([coauthors_df, new_row], ignore_index=True)

# Rimuove i doppioni e ordina per nome
coauthors_df.drop_duplicates(subset='author_id', inplace=True)
coauthors_df.sort_values(by='name', inplace=True)

# Salva il DataFrame in un file CSV
coauthors_df.to_csv('data/coauthors.csv', index=False)


#### 2b